In [14]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import AIMessage, HumanMessage, BaseMessage
from pydantic import BaseModel, Field
from typing import List, Sequence, Dict


In [15]:
class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    messages: List[BaseMessage] = Field(default_factory=list)

    def add_message(self, message: BaseMessage) -> None:
        self.messages.append(message)

    def add_messages(self, messages: Sequence[BaseMessage]) -> None:
        self.messages.extend(messages)

    def clear(self) -> None:
        self.messages.clear()



In [16]:
# Create an InMemoryHistory object and add messages
history = InMemoryHistory()
history.add_messages([HumanMessage(content="Hello!"), AIMessage(content="Hi, how can I help you?")])
history.add_messages((HumanMessage(content="I have a complaint."), AIMessage(content="Please describe your issue.")))

# Print the message history
for msg in history.messages:
    print(f"{msg.type}: {msg.content}")

human: Hello!
ai: Hi, how can I help you?
human: I have a complaint.
ai: Please describe your issue.


In [17]:
message_histories: Dict[str, BaseChatMessageHistory] = {}

def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
    """Retrieve or Create chat message history by session ID."""
    if session_id not in message_histories:
        message_histories[session_id] = InMemoryHistory()
    return message_histories[session_id]

In [45]:
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda
from dotenv import load_dotenv
import os

os.environ.pop("MISTRAL_API_KEY", None)
load_dotenv("./.env")

True

In [46]:

llm = ChatMistralAI(
    api_key=os.getenv("MISTRAL_API_KEY"),
    model_name="mistral-small-latest",
    max_retries=3
)

In [47]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful customer support assistant for an e-commerce platform."),
    ("user", "Below there is a complaint. Respond to that in a polite and helpful manner."),
    ("user", "Complaint: {complaint}"),
    ("user", "Message History:"),
    MessagesPlaceholder(variable_name="history")
])  # type: ignore

In [48]:
prompt.format_messages(
    complaint="I received the wrong item.",
    history=[
        HumanMessage(content="I want to return this item."),
        HumanMessage(content="Can you help me with the return process?")
    ]
)

[SystemMessage(content='You are a helpful customer support assistant for an e-commerce platform.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Below there is a complaint. Respond to that in a polite and helpful manner.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Complaint: I received the wrong item.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Message History:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I want to return this item.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Can you help me with the return process?', additional_kwargs={}, response_metadata={})]

In [49]:
# LCEL: Chain the prompt and LLM using | operator (LangChain Expression Language)
# Chain prompt and LLM
chain = prompt | llm

In [50]:
def process_complaint(input: dict) -> dict:
    session_id = input["session_id"]
    complaint = input["complaint"]
    history = get_by_session_id(session_id)

    response = chain.invoke({
        "complaint": complaint,
        "history": history.messages
    })

    return {"response": response.content}

In [ ]:
for chunk in process_complaint({
    "session_id": "session_123",
    "complaint": "I received a damaged product."
})["response"]:
    print(chunk, end="", flush=True)

{'response': "I'm really sorry to hear that you received a damaged product. I understand how frustrating this can be, and I'm here to help.\n\nTo assist you better, could you please provide me with the following details:\n\n1. Your order number or the email address associated with your order.\n2. A brief description of the damage or any photos you might have of the damaged product.\n\nOnce I have this information, I can look into the issue and guide you through the return or replacement process. We truly appreciate your patience and understanding.\n\nLooking forward to your response.\n\nBest,\n[Your Name]\nCustomer Support"}